In [1]:
import tensorflow as tf, cv2, numpy as np, matplotlib.pyplot as plt, plotly.express as px
import os

In [2]:
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [3]:
def image_getter(base_dir, skip_function):
    for part in os.listdir(base_dir):
        if skip_function(part):
            continue
        for img_class in os.listdir(os.path.join(base_dir,part)):
            for i in os.listdir(os.path.join(base_dir,part,img_class)):
                image = cv2.imread(os.path.join(base_dir,part,img_class,i),cv2.IMREAD_GRAYSCALE)
                image = np.reshape(image,image.shape+(1,))
                label=int(img_class.split('_')[0])-1
                if label>2:
                    label -=1
                if label>5:
                    label -=1
                yield image/255, label

def train_image_getter(base_dir="./../archive/leapgestrecog/leapGestRecog/", skip_dir='09'):
    return image_getter(base_dir, lambda x: skip_dir in x)
def test_image_getter(base_dir="./../archive/leapgestrecog/leapGestRecog/", leave_dir='09'):
    return image_getter(base_dir, lambda x: leave_dir not in x)

In [4]:
train = tf.data.Dataset.from_generator(train_image_getter, output_types=(tf.float32,tf.int8))
test = tf.data.Dataset.from_generator(test_image_getter, output_types=(tf.float32,tf.int8))
train_ds = train.shuffle(2000, reshuffle_each_iteration=True).repeat(1).batch(16)
test_ds = test.shuffle(2000, reshuffle_each_iteration=True).repeat(1).batch(16)
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False)

In [5]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(240, 640,1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding="same"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(64, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01),
                              activity_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.Dense(8, activation='softmax'))

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 240, 640, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 120, 320, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 320, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 60, 160, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 160, 256)      295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 30, 80, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 80, 512)       1

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['sparse_categorical_accuracy'])

In [8]:
history = model.fit(train_ds.as_numpy_iterator(), epochs=1, validation_data=test_ds.as_numpy_iterator())

      2/Unknown - 0s 124ms/step - loss: 3.3401 - sparse_categorical_accuracy: 0.0312WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0774s vs `on_train_batch_end` time: 0.1698s). Check your callbacks.
1125/1125 [==============================] - 270s 240ms/step - loss: 1.7459 - sparse_categorical_accuracy: 0.8902 - val_loss: 1.6874 - val_sparse_categorical_accuracy: 0.9085


In [9]:
model.evaluate(test_ds.as_numpy_iterator())

125/125 [==============================] - 9s 73ms/step - loss: 1.6874 - sparse_categorical_accuracy: 0.9085


[1.6873881816864014, 0.9085000157356262]

In [10]:
model.save("02_iteration_clahe.h5")